In [9]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

In [10]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\train\train\cat"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = img.resize((28,28))
            data.append(np.array(x))

        x_train = np.array(data)
        x_train = x_train.reshape(12500,28,28,3)
        return x_train

In [15]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\cat_%d.png" % epoch)
        plt.close()

In [16]:
dcgan = DCGAN()
dcgan.train(epochs=2000, batch_size=200, save_interval=100)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 14, 14, 32)        896       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 64)         

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.198010, acc.: 31.50%] [G loss: 0.705771]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.545241, acc.: 73.25%] [G loss: 1.336126]
2 [D loss: 0.540007, acc.: 71.50%] [G loss: 1.783971]
3 [D loss: 0.702454, acc.: 63.75%] [G loss: 2.091154]
4 [D loss: 0.786632, acc.: 54.75%] [G loss: 2.308234]
5 [D loss: 0.600105, acc.: 68.00%] [G loss: 2.049795]
6 [D loss: 0.520257, acc.: 76.00%] [G loss: 1.591835]
7 [D loss: 0.512618, acc.: 78.25%] [G loss: 1.083909]
8 [D loss: 0.545423, acc.: 70.50%] [G loss: 1.003952]
9 [D loss: 0.794528, acc.: 57.25%] [G loss: 1.101004]
10 [D loss: 0.757294, acc.: 55.50%] [G loss: 1.913139]
11 [D loss: 0.830403, acc.: 53.75%] [G loss: 2.178852]
12 [D loss: 0.693682, acc.: 67.25%] [G loss: 2.235893]
13 [D loss: 0.562335, acc.: 71.25%] [G loss: 1.454248]
14 [D loss: 0.342734, acc.: 86.00%] [G loss: 0.695716]
15 [D loss: 0.230986, acc.: 94.75%] [G loss: 0.370884]
16 [D loss: 0.204931, acc.: 95.75%] [G loss: 0.248797]
17 [D loss: 0.186265, acc.: 96.25%] [G loss: 0.301315]
18 [D loss: 0.168988, acc.: 96.75%] [G loss: 0.582438]
19 [D loss: 0.21993

...

1337 [D loss: 0.959847, acc.: 42.25%] [G loss: 0.670264]
1338 [D loss: 0.603534, acc.: 70.50%] [G loss: 0.979371]
1339 [D loss: 1.076935, acc.: 26.25%] [G loss: 0.695960]
1340 [D loss: 0.626843, acc.: 65.00%] [G loss: 0.611134]
1341 [D loss: 0.334875, acc.: 90.00%] [G loss: 0.682001]
1342 [D loss: 0.754899, acc.: 53.25%] [G loss: 0.513122]
1343 [D loss: 0.490944, acc.: 82.00%] [G loss: 0.617047]
1344 [D loss: 0.369504, acc.: 89.25%] [G loss: 0.759747]
1345 [D loss: 0.770644, acc.: 51.00%] [G loss: 0.674771]
1346 [D loss: 0.558748, acc.: 71.50%] [G loss: 0.534844]
1347 [D loss: 0.731139, acc.: 54.75%] [G loss: 0.608426]
1348 [D loss: 0.458320, acc.: 81.75%] [G loss: 1.078326]
1349 [D loss: 0.756830, acc.: 58.00%] [G loss: 0.727509]
1350 [D loss: 0.475004, acc.: 79.50%] [G loss: 0.428366]
1351 [D loss: 0.331256, acc.: 92.50%] [G loss: 0.397641]
1352 [D loss: 0.459104, acc.: 81.50%] [G loss: 0.445758]
1353 [D loss: 0.353235, acc.: 86.50%] [G loss: 0.306713]
1354 [D loss: 0.461203, acc.: 7

1484 [D loss: 0.800315, acc.: 43.75%] [G loss: 0.819852]
1485 [D loss: 0.688095, acc.: 56.00%] [G loss: 0.991424]
1486 [D loss: 0.578060, acc.: 69.25%] [G loss: 1.043316]
1487 [D loss: 0.741566, acc.: 56.00%] [G loss: 1.388604]
1488 [D loss: 0.748024, acc.: 54.25%] [G loss: 0.671496]
1489 [D loss: 0.404375, acc.: 87.00%] [G loss: 0.555040]
1490 [D loss: 0.694826, acc.: 59.50%] [G loss: 0.640237]
1491 [D loss: 0.591257, acc.: 69.75%] [G loss: 0.981598]
1492 [D loss: 0.632430, acc.: 65.50%] [G loss: 0.894674]
1493 [D loss: 1.003328, acc.: 25.50%] [G loss: 0.920312]
1494 [D loss: 0.941404, acc.: 33.25%] [G loss: 0.866553]
1495 [D loss: 0.594724, acc.: 67.75%] [G loss: 1.142687]
1496 [D loss: 0.536379, acc.: 73.75%] [G loss: 1.369871]
1497 [D loss: 0.716818, acc.: 55.50%] [G loss: 1.209261]
1498 [D loss: 0.478867, acc.: 80.25%] [G loss: 1.164758]
1499 [D loss: 0.794270, acc.: 47.25%] [G loss: 0.864905]
1500 [D loss: 0.449066, acc.: 84.25%] [G loss: 0.847565]
1501 [D loss: 0.471750, acc.: 8

1629 [D loss: 0.497214, acc.: 78.50%] [G loss: 0.642296]
1630 [D loss: 1.037104, acc.: 29.00%] [G loss: 0.891734]
1631 [D loss: 0.563881, acc.: 74.25%] [G loss: 0.887740]
1632 [D loss: 0.685418, acc.: 62.00%] [G loss: 0.895178]
1633 [D loss: 0.991345, acc.: 32.50%] [G loss: 0.882370]
1634 [D loss: 0.529016, acc.: 73.75%] [G loss: 0.907332]
1635 [D loss: 0.735951, acc.: 56.00%] [G loss: 1.295224]
1636 [D loss: 0.630230, acc.: 60.50%] [G loss: 1.152233]
1637 [D loss: 0.352225, acc.: 90.75%] [G loss: 1.084386]
1638 [D loss: 0.677797, acc.: 61.75%] [G loss: 0.833886]
1639 [D loss: 0.536601, acc.: 73.50%] [G loss: 1.049122]
1640 [D loss: 0.442239, acc.: 85.75%] [G loss: 1.443273]
1641 [D loss: 0.457448, acc.: 83.50%] [G loss: 0.761373]
1642 [D loss: 0.686369, acc.: 59.50%] [G loss: 1.007085]
1643 [D loss: 0.778206, acc.: 46.50%] [G loss: 0.724956]
1644 [D loss: 0.507589, acc.: 73.25%] [G loss: 0.972573]
1645 [D loss: 0.538591, acc.: 75.75%] [G loss: 1.009876]
1646 [D loss: 0.771075, acc.: 5

1773 [D loss: 0.796871, acc.: 47.50%] [G loss: 0.851428]
1774 [D loss: 0.415535, acc.: 86.75%] [G loss: 0.684461]
1775 [D loss: 0.502205, acc.: 80.25%] [G loss: 0.786838]
1776 [D loss: 0.551232, acc.: 75.25%] [G loss: 1.067839]
1777 [D loss: 0.682429, acc.: 57.75%] [G loss: 1.120119]
1778 [D loss: 0.699406, acc.: 55.50%] [G loss: 0.609284]
1779 [D loss: 0.856531, acc.: 40.75%] [G loss: 0.529268]
1780 [D loss: 0.513641, acc.: 78.25%] [G loss: 0.827320]
1781 [D loss: 0.544645, acc.: 76.25%] [G loss: 1.078518]
1782 [D loss: 0.563973, acc.: 72.50%] [G loss: 1.280508]
1783 [D loss: 0.605173, acc.: 69.25%] [G loss: 0.815658]
1784 [D loss: 0.630109, acc.: 64.75%] [G loss: 1.234275]
1785 [D loss: 0.361032, acc.: 91.75%] [G loss: 1.005332]
1786 [D loss: 0.633749, acc.: 62.75%] [G loss: 0.718169]
1787 [D loss: 0.487671, acc.: 78.00%] [G loss: 0.916425]
1788 [D loss: 0.575908, acc.: 70.75%] [G loss: 0.869401]
1789 [D loss: 0.497305, acc.: 79.00%] [G loss: 0.753285]
1790 [D loss: 0.684924, acc.: 6

1919 [D loss: 0.729753, acc.: 56.00%] [G loss: 0.788996]
1920 [D loss: 0.840555, acc.: 46.00%] [G loss: 0.842301]
1921 [D loss: 0.790205, acc.: 47.75%] [G loss: 0.591788]
1922 [D loss: 0.744228, acc.: 53.25%] [G loss: 0.875332]
1923 [D loss: 0.509199, acc.: 78.75%] [G loss: 1.030545]
1924 [D loss: 0.584160, acc.: 68.75%] [G loss: 1.260248]
1925 [D loss: 0.851353, acc.: 41.00%] [G loss: 0.664620]
1926 [D loss: 0.703500, acc.: 53.75%] [G loss: 1.941585]
1927 [D loss: 0.591053, acc.: 70.50%] [G loss: 1.205497]
1928 [D loss: 0.568301, acc.: 72.00%] [G loss: 0.900654]
1929 [D loss: 0.574807, acc.: 71.00%] [G loss: 1.363801]
1930 [D loss: 0.694665, acc.: 55.00%] [G loss: 0.924144]
1931 [D loss: 0.701160, acc.: 57.50%] [G loss: 1.014689]
1932 [D loss: 0.660724, acc.: 64.50%] [G loss: 0.937515]
1933 [D loss: 0.813592, acc.: 42.50%] [G loss: 0.942894]
1934 [D loss: 0.660605, acc.: 62.50%] [G loss: 0.988731]
1935 [D loss: 0.532147, acc.: 77.50%] [G loss: 0.682217]
1936 [D loss: 0.524061, acc.: 7